## 1. Importing Basic Libraries

In [1]:
import numpy as np
import pandas as pd

import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import string
import re


In [2]:
# Importing Dataset (.csv)

fake = pd.read_csv("./Dataset/Fake.csv")
true = pd.read_csv("./Dataset/True.csv")

# Creating labels for each category of fake news

fake["label"] = "fake"
true["label"] = "true"

fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


In [3]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


### Combining both datasets into one 

In [4]:
dataset = pd.concat([fake,true]).reset_index(drop = True)

# Combining title and text columns into one column "Article"

dataset['article'] = dataset['title'] + dataset['text']
dataset['article'].head()

# Removing unneded columns (maybe)
dataset.drop (['title', 'text', 'date','subject'], axis = 1, inplace =True)

dataset['article'].dropna(inplace = True)

dataset.head()

,label,article
0,fake,Donald Trump Sends Out Embarrassing New Year’...
1,fake,Drunk Bragging Trump Staffer Started Russian ...
2,fake,Sheriff David Clarke Becomes An Internet Joke...
3,fake,Trump Is So Obsessed He Even Has Obama’s Name...
4,fake,Pope Francis Just Called Out Donald Trump Dur...


In [5]:

'''

Merged dataset with counts of labelled news 
dropna = false shows the values that are empty

'''

dataset.label.value_counts(dropna = False)

fake    23481
true    21417
Name: label, dtype: int64

## 2. Dataset Preprocessing

### Converting to lowercase

In [6]:

dataset['cleaned'] = dataset['article'].apply(lambda word:word.lower())

## Rearranging dataset columns
column_names = ["article", "cleaned", "label"]
dataset = dataset.reindex(columns=column_names)

dataset.head()

,article,cleaned,label
0,Donald Trump Sends Out Embarrassing New Year’...,donald trump sends out embarrassing new year’...,fake
1,Drunk Bragging Trump Staffer Started Russian ...,drunk bragging trump staffer started russian ...,fake
2,Sheriff David Clarke Becomes An Internet Joke...,sheriff david clarke becomes an internet joke...,fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,trump is so obsessed he even has obama’s name...,fake
4,Pope Francis Just Called Out Donald Trump Dur...,pope francis just called out donald trump dur...,fake


### Removing Punctuations

In [7]:
def remove_punc(text):
    nopunc = "".join([a for a in text if a not in string.punctuation])
    return nopunc

In [8]:
dataset['cleaned'] = dataset['cleaned'].apply(lambda x: remove_punc(x))
dataset.head()

,article,cleaned,label
0,Donald Trump Sends Out Embarrassing New Year’...,donald trump sends out embarrassing new year’...,fake
1,Drunk Bragging Trump Staffer Started Russian ...,drunk bragging trump staffer started russian ...,fake
2,Sheriff David Clarke Becomes An Internet Joke...,sheriff david clarke becomes an internet joke...,fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,trump is so obsessed he even has obama’s name...,fake
4,Pope Francis Just Called Out Donald Trump Dur...,pope francis just called out donald trump dur...,fake


### Removing Numbers

In [9]:
def remove_num(text):
    nonum = "".join([i for i in text if not i.isdigit()])
    return nonum


In [10]:
dataset['cleaned'] = dataset['cleaned'].apply(lambda x: remove_num(x))
dataset.head()


,article,cleaned,label
0,Donald Trump Sends Out Embarrassing New Year’...,donald trump sends out embarrassing new year’...,fake
1,Drunk Bragging Trump Staffer Started Russian ...,drunk bragging trump staffer started russian ...,fake
2,Sheriff David Clarke Becomes An Internet Joke...,sheriff david clarke becomes an internet joke...,fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,trump is so obsessed he even has obama’s name...,fake
4,Pope Francis Just Called Out Donald Trump Dur...,pope francis just called out donald trump dur...,fake


### Tokenization

In [11]:
tokenizer = RegexpTokenizer('\w+')

In [12]:
dataset['cleaned'] = dataset['cleaned'].apply(lambda x: tokenizer.tokenize(x))
dataset['cleaned'].head(10)

0    [donald, trump, sends, out, embarrassing, new,...
1    [drunk, bragging, trump, staffer, started, rus...
2    [sheriff, david, clarke, becomes, an, internet...
3    [trump, is, so, obsessed, he, even, has, obama...
4    [pope, francis, just, called, out, donald, tru...
5    [racist, alabama, cops, brutalize, black, boy,...
6    [fresh, off, the, golf, course, trump, lashes,...
7    [trump, said, some, insanely, racist, stuff, i...
8    [former, cia, director, slams, trump, over, un...
9    [watch, brandnew, protrump, ad, features, so, ...
Name: cleaned, dtype: object

### Stopword Removal

In [13]:

stopwords = set(nltk.corpus.stopwords.words('english'))

def clean_stopwords(text):
    for word in text:
        if word in stopwords:
            text.remove(word)
    return text



In [14]:
dataset['cleaned'] = dataset['cleaned'].apply(lambda x: clean_stopwords(x))
dataset['cleaned'].head()

0    [donald, trump, sends, embarrassing, new, year...
1    [drunk, bragging, trump, staffer, started, rus...
2    [sheriff, david, clarke, becomes, internet, jo...
3    [trump, so, obsessed, even, obama, name, coded...
4    [pope, francis, called, donald, trump, christm...
Name: cleaned, dtype: object

In [15]:
#def pos_tags(text):
#   postag = nltk.pos_tag(text)
#   return postag

#dataset['postags'] = dataset['cleaned'].apply(lambda x: pos_tags(x))
#dataset['postags'].head()

### Meeting Notes 9/9/2020
* split dataset 70/30 and randomise 
* find possibility to merge classifiers 
* then compare it with another dataset 

9/9/2020

### Lemmatization

In [16]:
lemm = WordNetLemmatizer()

def lem_text(text):
    lem_text ="".join([lemm.lemmatize(w) for w in text])
    return lem_text

In [17]:
dataset['cleaned'] = dataset['cleaned'].apply(lambda x: lem_text(x))
dataset['cleaned'].head()

0    donaldtrumpsendsembarrassingnewyearevemessaged...
1    drunkbraggingtrumpstafferstartedrussiancollusi...
2    sheriffdavidclarkebecomesinternetjokethreateni...
3    trumpsoobsessedevenobamanamecodedwebsiteimages...
4    popefranciscalleddonaldtrumpchristmasspeechpop...
Name: cleaned, dtype: object

## 3. Split Test/Train data

In [18]:
# The dataset shall be then split into train/test data using sklearn 
# Manipulated variable = 'cleaned' column
# Responsiding variable = 'label' ccolumn

from sklearn.model_selection import train_test_split

x = dataset.cleaned 
y = dataset.label

xTrain,xTest,yTrain,yTest = train_test_split(x, y, test_size = 0.2, random_state = 42, shuffle = True)


## 4. Feature Extraction

### Generating POS tags

In [19]:
def pos_tags(text):
    postag = nltk.pos_tag(text)
    return postag

In [20]:
xTrainPOS = xTrain.apply(pos_tags)
print (xTrainPOS.head())

36335    [(t, NN), (u, JJ), (r, NN), (k, NN), (e, FW), ...
12384    [(b, NN), (r, NN), (e, VBZ), (a, DT), (k, NN),...
24419    [(t, NN), (r, NN), (u, JJ), (m, NN), (p, NN), ...
24740    [(s, VB), (a, DT), (u, JJ), (d, NN), (i, NN), ...
27039    [(t, NN), (r, NN), (u, JJ), (m, NN), (p, NN), ...
Name: cleaned, dtype: object


### TF-IDF



In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## Generating CountVectors

countVec = CountVectorizer()

xTrainCount = countVec.fit_transform(xTrain)
xTrainCount.shape

(35918, 31957)

In [27]:
## Generating TF-IDF vectors

tfIdf = TfidfTransformer()
xTrainTfIdf = tfIdf.fit_transform(xTrainCount)
xTrainTfIdf.shape

(35918, 31957)

## N-Grams

## Logistic Regression Modelling 

In [28]:
from sklearn.pipeline import Pipeline 

ml_pipeline = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

## SVM Modelling

In [ ]:


ml_pipeline = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])
